In [1]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))

from datetime import date, timedelta
import codecs
import pandas as pd

pd.set_option('display.max_columns', None)

from os.path import join

from bulletin import default_input, default_output
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols

from datetime import datetime


today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)


---------------------------------------------------------------

root: c:\SESA\dev\bulletin
parent: c:\SESA\dev
default_input: c:\SESA\dev\input
default_output: c:\SESA\dev\output

---------------------------------------------------------------



In [2]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [3]:
mb = Metabase(True)

limit: 100000
username: est.joaosilva@sesa.pr.gov.br
password: @Silva2020
Requesting session
Success code 200
Cookie:'431ea********-****-****-****-************11ae2'

sql_files:
	0: all_notifica
	1: coisa
	2: diario
	3: new_query
	4: old_diario
	5: usecols_notifica

sql_results:


In [6]:
mb.generate_notifica_query('query_linda')

Select 44 columns


"SELECT id, to_char(data_notificacao,'DD/MM/YYYY') AS data_notificacao, to_char(data_encerramento,'DD/MM/YYYY') AS data_encerramento, status_notificacao, origem, to_char(updated_at,'DD/MM/YYYY') AS updated_at, numero_do, classificacao_final, criterio_classificacao, evolucao, to_char(data_cura_obito,'DD/MM/YYYY') AS data_cura_obito, tipo_paciente, cns, cpf, passaporte, etnia, paciente, sexo, to_char(data_nascimento,'DD/MM/YYYY') AS data_nascimento, nome_mae, raca_cor, ocupacao, ocupacao_descricao, cod_cbo, escolaridade, pais_residencia, pais_municipio_residencia, uf_residencia, ibge_residencia, cep_residencia, logradouro_residencia, bairro_residencia, uf_unidade_notifica, ibge_unidade_notifica, to_char(data_cadastro,'DD/MM/YYYY') AS data_cadastro, to_char(data_coleta,'DD/MM/YYYY') AS data_coleta, to_char(data_recebimento,'DD/MM/YYYY') AS data_recebimento, to_char(data_liberacao,'DD/MM/YYYY') AS data_liberacao, requisicao, resultado, exame, metodo, lab_executor, to_char(data_1o_sintomas,

In [8]:
mb.list_sql_files()


sql_files:
	0: all_notifica
	1: coisa
	2: diario
	3: new_query
	4: old_diario
	5: query_linda
	6: usecols_notifica


In [10]:
notifica = mb.download_notificacao('query_linda', load=True)
# notifica = Notifica()
# notifica.load()

Download query_linda

Requesting csv)
Success code 200
Saving query in c:\SESA\dev\input\queries\query_linda_len.csv
Download finish!
time elapsed: 0:01:25.514803
Columns size: 1


query_size: 1150351

select ... limit 100000 offset 0

Requesting csv)
Success code 200
Saving query in c:\SESA\dev\input\queries\tmp\query_linda_0_100000.csv
Download finish!
time elapsed: 0:02:00.229215
Columns size: 44

select ... limit 100000 offset 100000

Requesting csv)
Success code 200
Saving query in c:\SESA\dev\input\queries\tmp\query_linda_100000_200000.csv
Download finish!
time elapsed: 0:03:05.865157
Columns size: 44

select ... limit 100000 offset 200000

Requesting csv)
Success code 200
Saving query in c:\SESA\dev\input\queries\tmp\query_linda_200000_300000.csv
Download finish!
time elapsed: 0:02:25.607799
Columns size: 44

select ... limit 100000 offset 300000

Requesting csv)
Success code 200
Saving query in c:\SESA\dev\input\queries\tmp\query_linda_300000_400000.csv
Download finish!
time el

In [11]:
notifica.save()

In [12]:
casos_notifica = notifica.df
casos_notifica

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_less,hash_more
0,66722,2020-04-13,2020-07-30,2.0,1.0,2020-08-19,None,2.0,1.0,1.0,2020-07-30,3.0,None,None,NaN,0.0,NILDA CASTORINA CORREIA CRUZ,2.0,1973-06-20,APARECIDA CORREIA DOS SANTOS,1.0,5.0,Atendente de panificadora de um supermercado,NaN,NaN,1.0,,41,412710,84267190,Rua Jaguatirica,Parque Limeira Área II,41,412710,2020-04-20,2020-04-13,2020-04-20,2020-04-20,202121000002,9.0,1,1,LAFFER,2020-03-30,2020-04-13,46,NILDACASTORINACORREIACRUZ46412710,NILDACASTORINACORREIACRUZ46412710,NILDACASTORINACORREIACRUZ45412710,NILDACASTORINACORREIACRUZ47412710
1,66776,2020-04-12,2020-05-02,2.0,1.0,2020-06-18,None,2.0,1.0,1.0,2020-05-02,3.0,None,029.654.009-90,NaN,0.0,ELIANA DA SILVA,2.0,1977-03-08,MARIA DAS GRAÇAS DE SOUZA,4.0,5.0,SETOR DE ABATEDOURO DESOSSA GTFOODS,NaN,NaN,1.0,,41,411810,87660000,RUA COLOMBIA,CENTRO,41,411810,2020-04-16,2020-04-16,2020-04-17,2020-04-19,201502005722,9.0,1,1,LABORATORIO CENTRAL DO ESTADO,NaT,2020-04-16,43,ELIANADASILVA43411810,ELIANADASILVA43411810,ELIANADASILVA42411810,ELIANADASILVA44411810
2,66777,2020-04-14,2021-03-23,2.0,1.0,2021-03-23,None,2.0,1.0,1.0,2020-12-16,3.0,None,018.903.649-40,NaN,NaN,SALETE LOURDES REBESCHINI DA SILVA,2.0,1952-06-24,NATALINA REBESCHINI,1.0,5.0,APOSENTADA,NaN,NaN,1.0,,41,412860,85585000,LINHA PILONETO,INTERIOR,41,412860,NaT,2020-12-14,NaT,NaT,-1,9.0,1,1,,2020-12-01,2020-12-14,67,SALETELOURDESREBESCHINIDASILVA67412860,SALETELOURDESREBESCHINIDASILVA67412860,SALETELOURDESREBESCHINIDASILVA66412860,SALETELOURDESREBESCHINIDASILVA68412860
3,66824,2020-04-13,2020-04-26,2.0,1.0,2020-08-18,None,2.0,2.0,1.0,2020-04-26,3.0,702601272873442,055.592.029-16,NaN,0.0,MARCIANO PEREIRA LUCAS,1.0,1985-07-04,LUIZA APARECIDA APARECIDA PEREIRA LUCAS,4.0,5.0,COLETOR DE OVOS,NaN,NaN,1.0,,41,410510,86630000,QUADRA 3 LOTE 17,JARDIM NAZARÉ,41,410510,NaT,NaT,NaT,NaT,-1,0.0,-1,3,,2020-04-12,2020-04-13,34,MARCIANOPEREIRALUCAS34410510,MARCIANOPEREIRALUCAS34410510,MARCIANOPEREIRALUCAS33410510,MARCIANOPEREIRALUCAS35410510
4,66864,2020-04-13,2020-04-29,2.0,1.0,2021-01-29,None,2.0,2.0,3.0,NaT,3.0,None,064.654.069-69,NaN,0.0,VALDOMIRA DA SILVA CORDEIRO,2.0,1935-10-05,DURVINA MARIA DA SILVA,99.0,0.0,Não Informado,0.0,NaN,1.0,,41,412625,NaN,NaN,NaN,41,412625,NaT,NaT,NaT,NaT,-1,0.0,-1,3,,2020-04-03,2020-04-13,84,VALDOMIRADASILVACORDEIRO84412625,VALDOMIRADASILVACORDEIRO84412625,VALDOMIRADASILVACORDEIRO83412625,VALDOMIRADASILVACORDEIRO85412625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50690,4189728,2021-06-11,NaT,1.0,15.0,NaT,None,2.0,1.0,NaN,2021-06-11,3.0,700008714758908,101.622.579-29,NaN,NaN,MURILO WENDLER TOZETTO,1.0,2007-01-22,CLAUDINEIA WENDLER TOZETTO,1.0,NaN,,NaN,NaN,1.0,,41,411010,84430000,RUA JOAO PUPO NETO,CENTRO,41,411010,NaT,NaT,NaT,NaT,-1,12.0,167,2,,2021-06-08,2021-06-11,14,MURILOWENDLERTOZETTO14411010,MURILOWENDLERTOZETTO14411010,MURILOWENDLERTOZETTO13411010,MURILOWENDLERTOZETTO15411010
50691,4189729,2021-06-11,NaT,1.0,1.0,2021-06-11,None,2.0,1.0,3.0,NaT,3.0,703407079640100,827.915.389-68,NaN,NaN,SUELI DE FATIMA PERASSOLI TEODORO,2.0,1970-05-18,DIRCE PERASSOLI,99.0,NaN,,NaN,NaN,1.0,,41,411420,86975000,NaN,NaN,41,411420,NaT,NaT,NaT,NaT,-1,12.0,167,2,FISIOCARE,2021-06-08,2021-06-11,51,SUELIDEFATIMAPERASSOLITEODORO51411420,SUELIDEFAT

In [14]:
schema = notifica.schema
identificacao_cols = schema.loc[schema['group_name']=='tb_pacientes','column'].values
identificacao_cols

array(['cns', 'cpf', 'passaporte', 'paciente', 'data_nascimento',
       'nome_mae'], dtype=object)

In [56]:
for col in casos_notifica:
    print(f"{col} -> {casos_notifica[col].isna().sum()}")

id -> 0
data_notificacao -> 49
data_encerramento -> 322637
status_notificacao -> 0
origem -> 0
updated_at -> 23189
numero_do -> 1125376
classificacao_final -> 0
criterio_classificacao -> 0
evolucao -> 110375
data_cura_obito -> 453680
tipo_paciente -> 19
cns -> 102251
cpf -> 159345
passaporte -> 1146568
etnia -> 696487
paciente -> 0
sexo -> 0
data_nascimento -> 54
nome_mae -> 1875
raca_cor -> 3719
ocupacao -> 573005
ocupacao_descricao -> 5553
cod_cbo -> 673176
escolaridade -> 1074103
pais_residencia -> 0
pais_municipio_residencia -> 12
uf_residencia -> 0
ibge_residencia -> 0
cep_residencia -> 367124
logradouro_residencia -> 126033
bairro_residencia -> 234720
uf_unidade_notifica -> 0
ibge_unidade_notifica -> 0
data_cadastro -> 285525
data_coleta -> 120927
data_recebimento -> 399705
data_liberacao -> 333255
requisicao -> 0
resultado -> 3754
exame -> 0
metodo -> 0
lab_executor -> 0
data_1o_sintomas -> 191932
data_diagnostico -> 9
idade -> 0
hash -> 0
hash_atend -> 0
hash_less -> 0
hash_mor

In [54]:
casos = casos_notifica.groupby(['hash'],dropna=False)[['id']].count()

In [55]:
casos

,id
nome_mae,
,15262
\tVILMA AMARO NASCIMENTO,1
,10
IVANIA CONSTATINO,1
JISELI CRISTINA LEANDRO DOS SANTOS,1
...,...
ZÉLIA FERNANDES DA SILVA,1
ZÉLIA RODRIGUES,1
ÁGATHA GRAÇA,1


In [ ]:
exit()
date_parser = lambda x: pd.to_datetime(x, errors='coerce', format='%d/%m/%Y')

notifica = pd.read_csv(diario,
                       sep=',',
                       converters = {
                           'paciente': normalize_text,
                           'origem': lambda x: normalize_number(x,fill=0),
                           'nome_mae': normalize_text,
                           'idade': lambda x: normalize_number(x,fill=0),
                           'pais_residencia': lambda x: normalize_number(x,fill=0),
                           'uf_residencia': lambda x: normalize_number(x,fill=0),
                           'ibge_residencia': lambda x: normalize_number(x,fill=999999),
                           'uf_unidade_notifica': lambda x: normalize_number(x,fill=0),
                           'ibge_unidade_notifica': lambda x: normalize_number(x,fill=999999),
                           'criterio_classificacao': lambda x: normalize_number(x,fill=0),
                           'evolucao': lambda x: normalize_number(x,fill=3) }
                       )


notifica['data_cura_obito'] = notifica['data_cura_obito'].apply(date_parser)
notifica['data_liberacao'] = notifica['data_liberacao'].apply(date_parser)

notifica.loc[notifica['evolucao']==5, 'evolucao'] = 2
notifica.loc[~notifica['evolucao'].isin([1,2,3]),'evolucao'] = 3

notifica['hash'] = (notifica['paciente'].apply(normalize_hash) +
                  notifica['idade'].astype(str) +
                  notifica['ibge_residencia'].astype(str) )


notifica['hash_atend'] = (notifica['paciente'].apply(normalize_hash) +
                  notifica['idade'].astype(str) +
                  notifica['ibge_unidade_notifica'].astype(str) )

notifica['hash_less'] = ( notifica['paciente'].apply(normalize_hash) +
                        notifica['idade'].apply(lambda x: str(x - 1)) +
                        notifica['ibge_residencia'].astype(str) )

notifica['hash_more'] = ( notifica['paciente'].apply(normalize_hash) +
                        notifica['idade'].apply(lambda x: str(x + 1)) +
                        notifica['ibge_residencia'].astype(str) )

notifica['hash_diag'] = notifica['paciente'].apply(normalize_hash) + notifica['data_liberacao'].apply(date_hash)

notifica = pd.merge(notifica.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','rs','mun_resid','uf','municipio']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
notifica = pd.merge(notifica.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})

notifica = notifica.loc[~(
    (notifica['data_liberacao'].isna()) |
    (notifica['sexo'].isna())
)]

notifica = notifica.loc[(
    (notifica['data_liberacao'] < pd.to_datetime(datetime.today())) &
    (notifica['data_liberacao'] > pd.to_datetime(datetime.strptime('12/03/2020','%d/%m/%Y')))
)]

notifica.groupby('evolucao')[['id']].count()

In [5]:
cc = CasosConfirmados()
cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}")
casosct = cc.df
casosct.tail()

databases: ['cc_21_05_2021', 'cc_22_05_2021', 'cc_23_05_2021', 'cc_25_05_2021', 'cc_27_05_2021', 'cc_28_05_2021', 'cc_29_05_2021']
Load /home/vinnylg/projects/covid19datascience/bulletin/database/casos_confirmados/cc_29_05_2021.pkl
loading Casos Confirmados from pkl time elepsed:00:00:24


,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_atend,hash_less,hash_more,hash_diag
1089772,1182013,3680672.0,PR,411840,411840,ZILDA APARECIDA BARBOZA BATISTA,F,58,"COVID-19, Biologia Molecular",2021-05-18,2021-05-29,2021-05-11,1.0,2021-05-28 00:00:00,NaT,ZILDAAPARECIDABARBOZABATISTA58411840,ZILDAAPARECIDABARBOZABATISTA58411840,ZILDAAPARECIDABARBOZABATISTA57411840,ZILDAAPARECIDABARBOZABATISTA59411840,ZILDAAPARECIDABARBOZABATISTA18052021
1089773,1182014,3777427.0,PR,410690,410690,ZILDA MEDEIROS DOS SANTOS,F,61,"COVID-19, Biologia Molecular",2021-05-22,2021-05-29,2021-05-13,3.0,NaT,NaT,ZILDAMEDEIROSDOSSANTOS61410690,ZILDAMEDEIROSDOSSANTOS61410690,ZILDAMEDEIROSDOSSANTOS60410690,ZILDAMEDEIROSDOSSANTOS62410690,ZILDAMEDEIROSDOSSANTOS22052021
1089774,1182015,3923416.0,PR,410850,410850,ZILTON ANTONIO LEITE,M,31,Teste Rápido Antígeno (Imunocromatografia),2021-05-29,2021-05-29,2021-05-23,3.0,NaT,NaT,ZILTONANTONIOLEITE31410850,ZILTONANTONIOLEITE31410850,ZILTONANTONIOLEITE30410850,ZILTONANTONIOLEITE32410850,ZILTONANTONIOLEITE29052021
1089775,1182016,3861326.0,PR,411915,999999,ZORAIDE ESTEFANO DA SILVA,F,71,"COVID-19, Biologia Molecular",2021-05-27,2021-05-29,2021-05-23,3.0,NaT,NaT,ZORAIDEESTEFANODASILVA71411915,ZORAIDEESTEFANODASILVA71999999,ZORAIDEESTEFANODASILVA70411915,ZORAIDEESTEFANODASILVA72411915,ZORAIDEESTEFANODASILVA27052021
1089776,1182017,3900570.0,PR,411520,411520,ZULEIDE PEREIRA DE CARVALHO,F,47,"COVID-19, Biologia Molecular",2021-05-28,2021-05-29,2021-05-20,3.0,NaT,NaT,ZULEIDEPEREIRADECARVALHO47411520,ZULEIDEPEREIRADECARVALHO47411520,ZULEIDEPEREIRADECARVALHO46411520,ZULEIDEPEREIRADECARVALHO48411520,ZULEIDEPEREIRADECARVALHO28052021


In [6]:
novos_casos = notifica.loc[~(
    (notifica['hash'].isin(casosct['hash'])) |
    (notifica['hash'].isin(casosct['hash_less'])) |
    (notifica['hash'].isin(casosct['hash_more'])) |
    (notifica['hash_diag'].isin(casosct['hash_diag']))
)].copy()

novos_casos['data_com'] = pd.to_datetime(date.today()).strftime('%d/%m/%Y')
novos_casos = novos_casos.sort_values('paciente')
novos_casos

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend,data_com
56559,3882465,1,26/05/2021,29/05/2021,ABEL DE OLIVEIRA SIEBRE,M,LAURINDA FONSECA DE OLIVEIRA,46,09/12/1974,1,41,410930,41,410930,"COVID-19, Biologia Molecular",1,2021-05-29,26/05/2021,28/05/2021,18/05/2021,3,NaT,ABELDEOLIVEIRASIEBRE46410930,ABELDEOLIVEIRASIEBRE46410930,ABELDEOLIVEIRASIEBRE45410930,ABELDEOLIVEIRASIEBRE47410930,ABELDEOLIVEIRASIEBRE29052021,10,GUARANIACU,PR,Guaraniaçu,GUARANIACU,30/05/2021
56153,3880493,1,26/05/2021,29/05/2021,ABRAAO VALDECIR DA SILVA,M,EVA CANDIDA DA SILVA,45,16/05/1976,1,41,410930,41,410930,"COVID-19, Biologia Molecular",1,2021-05-29,26/05/2021,28/05/2021,21/05/2021,3,NaT,ABRAAOVALDECIRDASILVA45410930,ABRAAOVALDECIRDASILVA45410930,ABRAAOVALDECIRDASILVA44410930,ABRAAOVALDECIRDASILVA46410930,ABRAAOVALDECIRDASILVA29052021,10,GUARANIACU,PR,Guaraniaçu,GUARANIACU,30/05/2021
70715,3950014,1,29/05/2021,29/05/2021,ACIR RIBEIRO,M,IRACEMA RIBEIRO,40,17/12/1980,1,41,411125,41,411125,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-29,29/05/2021,NaN,17/05/2021,3,NaT,ACIRRIBEIRO40411125,ACIRRIBEIRO40411125,ACIRRIBEIRO39411125,ACIRRIBEIRO41411125,ACIRRIBEIRO29052021,2,ITAPERUCU,PR,Itaperuçu,ITAPERUCU,30/05/2021
68982,3941757,1,28/05/2021,29/05/2021,ADAEL BREVINSKI ARAGAO,M,TEREZA BREVINSKI,23,12/11/1997,1,41,411370,41,411370,"COVID-19, Biologia Molecular",1,2021-05-24,23/05/2021,24/05/2021,NaN,3,NaT,ADAELBREVINSKIARAGAO23411370,ADAELBREVINSKIARAGAO23411370,ADAELBREVINSKIARAGAO22411370,ADAELBREVINSKIARAGAO24411370,ADAELBREVINSKIARAGAO24052021,17,LONDRINA,PR,Londrina,LONDRINA,30/05/2021
53145,3869500,1,26/05/2021,29/05/2021,ADAN FELIPE CARDOSO CANDIDO,M,NEIDE MARIA SILVA CARDOSO,28,24/02/1993,1,41,411885,41,411885,"COVID-19, Biologia Molecular",1,2021-05-28,26/05/2021,27/05/2021,22/05/2021,3,NaT,ADANFELIPECARDOSOCANDIDO28411885,ADANFELIPECARDOSOCANDIDO28411885,ADANFELIPECARDOSOCANDIDO27411885,ADANFELIPECARDOSOCANDIDO29411885,ADANFELIPECARDOSOCANDIDO28052021,12,PEROBAL,PR,Perobal,PEROBAL,30/05/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54597,3875084,1,26/05/2021,29/05/2021,ZENI BECKER,F,MARIA ANTONIA CZARNESKY,68,16/08/1952,1,41,412760,41,412760,"COVID-19, Biologia Molecular",1,2021-05-27,25/05/2021,27/05/2021,25/05/2021,3,NaT,ZENIBECKER68412760,ZENIBECKER68412760,ZENIBECKER67412760,ZENIBECKER69412760,ZENIBECKER27052021,2,TIJUCAS DO SUL,PR,Tijucas do Sul,TIJUCAS DO SUL,30/05/2021
70336,3948238,1,29/05/2021,29/05/2021,ZENILDA APARECIDA SANCHEZ PENTEADO,F,MARIA AUGUSTA BUENO SANCHEZ,65,27/08/1955,1,41,410465,41,410465,Teste Rápido IgM/IgG não discriminatório (Imun...,1,2021-05-29,29/05/2021,29/05/2021,20/05/2021,3,NaT,ZENILDAAPARECIDASANCHEZPENTEADO65410465,ZENILDAAPARECIDASANCHEZPENTEADO65410465,ZENILDAAPARECIDASANCHEZPENTEADO64410465,ZENILDAAPARECIDASANCHEZPENTEADO66410465,ZENILDAAPARECIDASANCHEZPENTEADO29052021,3,CARAMBEI,PR,Carambeí,CARAMBEI,30/05/2021
70453,3948811,1,29/05/2021,29/05/2021,ZENIR GOMES RODRIGUES,F,ELVIRA GOMES,42,01/11/1978,1,41,410895,41,410895,Teste Rápido IgM (Imunocromatografia),1,2021-05-28,28/05/2021,28/05/2021,28/05/2021,3,NaT,ZENIRGOMESRODRIGUES42410895,ZENIRGOMESRODRIGUES42410895,ZENIRGOMESRODRIGUES41410895,ZENIRGOMESRODRIGUES43410895,ZENIRGOMESRODRIGUES28052021,4,GUAMIRANGA,PR,Guamiranga,GUAMIRANGA,30/05/2021
70879,3950797,1,29/05/2021,29/05/2021,ZENITE MARIA DOS SANTOS SILVA,F,ORESMIDIA MARIA DA CONCEICAO,75,01/04/1946,1,41,410830,41,410830,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-29,29/05/2021,29/05/2021,24/05/2021,3,NaT,ZENITEMARIADOSSANTOSSILVA75410830,ZENITEMARIADOSSANTOSSILVA75410830,ZENITEMARIAD

In [7]:
novos_casos_errados = novos_casos.loc[(
    (novos_casos['data_liberacao'] <= pd.to_datetime('2020-03-01')) |
    (novos_casos['data_liberacao'] > pd.to_datetime(date.today()))
)]

novos_casos = novos_casos.loc[~(
    (novos_casos['data_liberacao'] <= pd.to_datetime('2020-03-01')) |
    (novos_casos['data_liberacao'] > pd.to_datetime(date.today()))
)]

novos_casos[['id','paciente','sexo','idade','mun_resid', 'mun_atend', 'rs', 'exame','data_liberacao','data_com','data_1o_sintomas','hash']].to_excel(join(default_output,'novos_casos.xlsx'))

novos_casos_errados

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend,data_com


In [8]:
obitos_notifica = notifica.loc[notifica['evolucao']==2]
obitos_casos = casosct.loc[casosct['evolucao']==2]

novos_obitos = obitos_notifica.loc[~(
    (obitos_notifica['hash'].isin(obitos_casos['hash'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_less'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_more'])) |
    (obitos_notifica['hash_diag'].isin(obitos_casos['hash_diag'])) 
)]

novos_obitos = novos_obitos.loc[novos_obitos['data_cura_obito'].notna()]
novos_obitos = novos_obitos.loc[novos_obitos['hash'].isin(casosct['hash'].tolist() + novos_casos['hash'].tolist())]
novos_obitos = novos_obitos.sort_values('paciente')
novos_obitos

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend
22493,3702179,1,17/05/2021,29/05/2021,ALEVINO ALVES,M,JURACI MENDES ALVES,53,17/03/1968,1,41,411760,41,411760,"COVID-19, Biologia Molecular",1,2021-05-22,17/05/2021,22/05/2021,11/05/2021,2,2021-05-29,ALEVINOALVES53411760,ALEVINOALVES53411760,ALEVINOALVES52411760,ALEVINOALVES54411760,ALEVINOALVES22052021,7,PALMAS,PR,Palmas,PALMAS
47177,3847898,1,25/05/2021,29/05/2021,ANTONIO CARLOS FONSECA,M,ALZENIRA FONZECA,47,03/05/1974,1,41,412855,41,412855,Vírus Respiratório,1,2021-05-28,26/05/2021,28/05/2021,20/05/2021,2,2021-05-29,ANTONIOCARLOSFONSECA47412855,ANTONIOCARLOSFONSECA47412855,ANTONIOCARLOSFONSECA46412855,ANTONIOCARLOSFONSECA48412855,ANTONIOCARLOSFONSECA28052021,10,VERA CRUZ DO OESTE,PR,Vera Cruz do Oeste,VERA CRUZ DO OESTE
8049,3362466,1,21/04/2021,29/05/2021,ANTONIO JUKOWSKI,M,SEBASTIANA SCHOF JUKOWSKI,71,13/01/1950,1,41,410940,41,410940,"COVID-19, Biologia Molecular",1,2021-04-26,23/04/2021,26/04/2021,20/04/2021,2,2021-05-28,ANTONIOJUKOWSKI71410940,ANTONIOJUKOWSKI71410940,ANTONIOJUKOWSKI70410940,ANTONIOJUKOWSKI72410940,ANTONIOJUKOWSKI26042021,5,GUARAPUAVA,PR,Guarapuava,GUARAPUAVA
12789,3589341,1,10/05/2021,29/05/2021,BENI ALEXANDRE DELFINO DE PAULA,F,DORVIRIA RODRIGUES DELFINO,57,23/04/1964,1,41,410940,41,410940,Vírus Respiratório,1,2021-05-11,09/05/2021,11/05/2021,29/04/2021,2,2021-05-28,BENIALEXANDREDELFINODEPAULA57410940,BENIALEXANDREDELFINODEPAULA57410940,BENIALEXANDREDELFINODEPAULA56410940,BENIALEXANDREDELFINODEPAULA58410940,BENIALEXANDREDELFINODEPAULA11052021,5,GUARAPUAVA,PR,Guarapuava,GUARAPUAVA
12693,3587852,1,10/05/2021,29/05/2021,CLAUDIO POHL,M,ROSELI GONCALVES POHL,44,07/11/1976,1,41,410940,41,410940,"COVID-19, Biologia Molecular",1,2021-05-13,10/05/2021,12/05/2021,03/05/2021,2,2021-05-28,CLAUDIOPOHL44410940,CLAUDIOPOHL44410940,CLAUDIOPOHL43410940,CLAUDIOPOHL45410940,CLAUDIOPOHL13052021,5,GUARAPUAVA,PR,Guarapuava,GUARAPUAVA
9869,3484920,1,03/05/2021,29/05/2021,CLEUDE JORGE WIRTH,M,HELGA LUIZA WUELF WIRTH,52,10/09/1968,1,41,412520,41,412520,Vírus Respiratório,1,2021-05-14,12/05/2021,14/05/2021,01/05/2021,2,2021-05-29,CLEUDEJORGEWIRTH52412520,CLEUDEJORGEWIRTH52412520,CLEUDEJORGEWIRTH51412520,CLEUDEJORGEWIRTH53412520,CLEUDEJORGEWIRTH14052021,8,SAO JORGE D'OESTE,PR,São Jorge d'Oeste,SAO JORGE D'OESTE
49033,3854434,1,25/05/2021,29/05/2021,DERCY ZEPERLIN GUEDES,F,ANTONIA REVELI ZAMPERTIN,83,14/02/1938,1,41,411180,41,411180,"COVID-19, Biologia Molecular",1,2021-05-24,21/05/2021,24/05/2021,16/05/2021,2,2021-05-29,DERCYZEPERLINGUEDES83411180,DERCYZEPERLINGUEDES83411180,DERCYZEPERLINGUEDES82411180,DERCYZEPERLINGUEDES84411180,DERCYZEPERLINGUEDES24052021,19,JACAREZINHO,PR,Jacarezinho,JACAREZINHO
12700,3587899,1,10/05/2021,29/05/2021,DIRCEU NARDI,M,DELVINA SORDI NARDI,61,13/11/1959,1,41,410100,41,410100,Vírus Respiratório,1,2021-05-21,18/05/2021,21/05/2021,06/05/2021,2,2021-05-29,DIRCEUNARDI61410100,DIRCEUNARDI61410100,DIRCEUNARDI60410100,DIRCEUNARDI62410100,DIRCEUNARDI21052021,8,AMPERE,PR,Ampére,AMPERE
29177,3761400,1,20/05/2021,29/05/2021,IVONETE DE FATIMA ANDRETTI ALVES,F,MARIA DE LURDES ANDRETTI,49,27/11/1971,1,41,410940,41,410940,Vírus Respiratório,1,2021-05-25,21/05/2021,24/05/2021,03/05/2021,2,2021-05-28,IVONETEDEFATIMAANDRETTIALVES49410940,IVONETEDEFATIMAANDRETTIALVES49410940,IVONETEDEFATIMAANDRETTIALVES48410940,IVONETEDEFATIMAANDRETTIALVES50410940,IVONETEDEFATIMAANDRETTIALVES25052021,5,GUARAPUAVA,PR,Guarapuava,GUARAPUAVA
34806,3798791,1,21/05/2021,29/05/2021,JACIRA GABRIEL DE SOUZA RAMOS,F,ELVIRA DE ASSUNCAO DE SOUZA,55,08/06/1965,1,41,410940,41,410940,Vírus Respiratório,1,2021-05-28,27/05/2021,28/05/2021,17/05/2021,2,2021-05-27,JACIRAGABRIELDESOUZARAMOS55

In [9]:
novos_obitos_errados = novos_obitos.loc[(
    (novos_obitos['data_cura_obito'] <= pd.to_datetime('2020-03-01')) |
    (novos_obitos['data_cura_obito'] > pd.to_datetime(date.today()))
)]

novos_obitos = novos_obitos.loc[~(
    (novos_obitos['data_cura_obito'] <= pd.to_datetime('2020-03-01')) |
    (novos_obitos['data_cura_obito'] > pd.to_datetime(date.today()))
)]

novos_obitos[['id','paciente','sexo','idade','mun_resid', 'rs', 'data_cura_obito','hash']].to_excel(join(default_output,'novos_obitos.xlsx'))

novos_obitos_errados

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend
10906,3535459,1,05/05/2021,28/05/2021,PEDRO RIBEIRO MARTINS,M,EMILIA RIBEIRO MARTINS,70,12/08/1950,1,41,411010,41,411990,Vírus Respiratório,1,2021-05-10,06/05/2021,07/05/2021,19/04/2021,2,2021-08-28,PEDRORIBEIROMARTINS70411010,PEDRORIBEIROMARTINS70411990,PEDRORIBEIROMARTINS69411010,PEDRORIBEIROMARTINS71411010,PEDRORIBEIROMARTINS10052021,4,IMBITUVA,PR,Imbituva,PONTA GROSSA


In [10]:
casos_confirmados = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios[['ibge','rs']],how='left',on='ibge')

casos_confirmadosPR = casos_confirmados.loc[casos_confirmados['rs']!=99]

obitos_confirmados =  casos_confirmados.loc[casos_confirmados['evolucao']==2]

obitos_confirmadosPR = obitos_confirmados.loc[obitos_confirmados['rs']!=99]

print(f"Total de casos: {len(casos_confirmados)} + {len(novos_casos)}")
print(f"Total de obitos: {len(obitos_confirmados)} + {len(novos_obitos)}\n\n")

novos_casosPR = novos_casos.loc[novos_casos['rs']!=99].copy()
print(f"Total de casos PR: {len(casos_confirmadosPR)} + {len(novos_casosPR)}")

novos_obitosPR = novos_obitos.loc[novos_obitos['rs']!=99].copy()
print(f"Total de obitos PR: {len(obitos_confirmadosPR)} + {len(novos_obitosPR)}")

novos_casosFora = novos_casos.loc[novos_casos['rs']==99].copy()
print(f"Total de casos Fora: {len(casos_confirmados) - len(casos_confirmadosPR)} + {len(novos_casosFora)}")

novos_obitosFora = novos_obitos.loc[novos_obitos['rs']==99].copy()
print(f"Total de obitos Fora: {len(obitos_confirmados) - len(obitos_confirmadosPR)} + {len(novos_obitosFora)}")


novos_obitosPR_group = novos_obitosPR.groupby(by='municipio')

data_retroativos = ontem - timedelta(14)

retroativos = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) <= data_retroativos)].sort_values(by='data_liberacao')
last2weeks = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) > data_retroativos) & (novos_casosPR['data_liberacao'].apply(date_parser) <= anteontem)].sort_values(by='data_liberacao')
hoje = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) > anteontem)].sort_values(by='data_liberacao')

obitos_retroativos = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) <= data_retroativos)].sort_values(by='data_cura_obito')
obitos_last2weeks = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > data_retroativos) & (novos_obitosPR['data_cura_obito'].apply(date_parser) <= anteontem)].sort_values(by='data_cura_obito')
obitos_hoje = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > anteontem)].sort_values(by='data_cura_obito')

relatorio_file = join(default_output,f"relatorio_{(today.strftime('%d/%m/%Y_%Hh').replace('/','_').replace(' ',''))}.txt")
with codecs.open(relatorio_file,"w","utf-8-sig") as relatorio:
    relatorio.write(f"{today.strftime('%d/%m/%Y')}\n")
    relatorio.write(f"{len(novos_casosPR):,} novos casos residentes ".replace(',','.'))

    if len(novos_casosFora) > 0:
        relatorio.write(f"e {len(novos_casosFora):,} não residente{'s' if len(novos_casosFora) > 1 else ''} ".replace(',','.'))
    relatorio.write(f"divulgados no PR.\n")

    relatorio.write(f"{len(casos_confirmadosPR)+len(novos_casosPR):,} casos confirmados residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(casos_confirmados)+len(novos_casos):,} total geral.\n\n".replace(',','.'))
    relatorio.write(f"{len(novos_obitosPR):,} Óbitos residentes do PR:\n".replace(',','.'))

    for municipio, obitos in novos_obitosPR_group:
        relatorio.write(f"{len(obitos):,} {municipio}\n".replace(',','.'))

    if len(novos_obitosFora) > 0:
        relatorio.write('\n')
        relatorio.write(f"{len(novos_obitosFora):,} Óbito{'s' if len(novos_obitosFora) > 1 else ''} não residente{'s' if len(novos_obitosFora) > 1 else ''} do PR.\n".replace(',','.'))

    relatorio.write('\n')
    relatorio.write(f"{len(obitos_confirmadosPR)+len(novos_obitosPR):,} óbitos residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmados)+len(novos_obitos):,} total geral.\n\n".replace(',','.'))

    for _, row in novos_obitos.iterrows():
       relatorio.write(f"{row['sexo']}\t{row['idade']}\t{row['municipio'] if row['rs']!=99 else (row['municipio']+'/'+row['uf'])}\t{str(row['rs']).zfill(2) if row['rs']!=99 else '#N/D'}\t{row['data_cura_obito'].day}/{static.meses[row['data_cura_obito'].month-1]}/{row['data_cura_obito'].year}\n")
    relatorio.write('\n')

    if True:

        #casos
        relatorio.write(f"{len(novos_casosPR):,} novos casos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(retroativos):,} casos retroativos confirmados no período de {retroativos.iloc[0]['data_liberacao'].strftime('%d/%m/%Y')} à {retroativos.iloc[-1]['data_liberacao'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(last2weeks):,} novos casos confirmados no período de {last2weeks.iloc[0]['data_liberacao'].strftime('%d/%m/%Y')} à {last2weeks.iloc[-1]['data_liberacao'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(hoje):,} novos casos confirmados hoje.\n\n".replace(',','.'))


        novos_casosPR['month'] = novos_casosPR.apply(lambda x: x['data_liberacao'].month, axis=1)
        novos_casosPR['year'] = novos_casosPR.apply(lambda x: x['data_liberacao'].year, axis=1)
        relatorio.write('Novos casos por meses:\n')
        for group, value in novos_casosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        #obitos
        relatorio.write(f"{len(novos_obitosPR):,} novos obitos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(obitos_retroativos):,} obitos retroativos ocorridos no período de {obitos_retroativos.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_retroativos.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_last2weeks):,} novos obitos ocorridos no período de {obitos_last2weeks.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_last2weeks.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_hoje):,} novos obitos ocorridos hoje.\n\n".replace(',','.'))

        relatorio.write('Novos obitos por meses:\n')
        novos_obitosPR['month'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].month, axis=1)
        novos_obitosPR['year'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].year, axis=1)
        for group, value in novos_obitosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        relatorio.write('Novos obitos por dia:\n')
        for group, value in novos_obitosPR.groupby(by='data_cura_obito'):
            relatorio.write(f"{group.strftime('%d/%m/%Y')}: {len(value)}\n")

        #-----RELATÓRIO DA COMUNICAÇÃO--------------
        obitos_list = []
        munic = []
        for municipio, obitos in novos_obitosPR_group:
            obito = len(obitos)
            obitos_list.append(obito)
            munic.append(municipio)             

        dicionario = (dict(zip(list(munic),list(obitos_list))))
        #print(dicionario)
        dicionario = sorted(dicionario.items(),key=lambda x: x[1], reverse = True)
        #print(dicionario)


        relatorio.write(f"\nOs pacientes que foram a óbito residiam em: ")
        for municip, obit in dict(dicionario).items():
            if obit != 1:
                relatorio.write(f"{municip} ({obit})")
                relatorio.write(f", ")
        relatorio.write(f".\n")
        relatorio.write(f"A Sesa registra ainda a morte de uma pessoa que residia em cada um dos seguintes municípios:")
        for municip, obit in dict(dicionario).items():
            if obit == 1:
                relatorio.write(f"{municip}")
                relatorio.write(f", ")


Total de casos: 1089777 + 1293
Total de obitos: 26370 + 21


Total de casos PR: 1083889 + 1289
Total de obitos PR: 26222 + 20
Total de casos Fora: 5888 + 4
Total de obitos Fora: 148 + 1


IndexError: single positional indexer is out-of-bounds

In [11]:
with codecs.open(relatorio_file,"r","utf-8-sig") as relatorio:
    print(relatorio.read())

30/05/2021
1.289 novos casos residentes e 4 não residentes divulgados no PR.
1.085.178 casos confirmados residentes do PR.
1.091.070 total geral.

20 Óbitos residentes do PR:
1 Ampére
8 Guarapuava
1 Guaíra
1 Jacarezinho
1 Jandaia do Sul
1 Mariluz
1 Maringá
1 Palmas
1 Rio Azul
1 Santa Tereza do Oeste
1 São Jorge d'Oeste
1 São José dos Pinhais
1 Vera Cruz do Oeste

1 Óbito não residente do PR.

26.242 óbitos residentes do PR.
26.391 total geral.

M	53	Palmas	07	29/mai/2021
M	47	Vera Cruz do Oeste	10	29/mai/2021
M	71	Guarapuava	05	28/mai/2021
F	57	Guarapuava	05	28/mai/2021
M	44	Guarapuava	05	28/mai/2021
M	52	São Jorge d'Oeste	08	29/mai/2021
F	83	Jacarezinho	19	29/mai/2021
M	61	Ampére	08	29/mai/2021
F	49	Guarapuava	05	28/mai/2021
F	55	Guarapuava	05	27/mai/2021
M	58	Guarapuava	05	28/mai/2021
M	73	Jandaia do Sul	16	29/mai/2021
F	62	Guarapuava	05	28/mai/2021
M	28	Maringá	15	29/mai/2021
M	90	Guaíra	20	29/mai/2021
F	89	Rio Azul	04	29/mai/2021
F	54	Mariluz	12	29/mai/2021
F	63	Itaquiraí/MS	#N/D	2

In [ ]:
while input('bateu?') != 'bateu': pass

In [ ]:
novos_casos = novos_casos.rename(columns={
    'id':'id_notifica',
    'ibge_residencia': 'ibge_resid',
    'ibge_unidade_notifica': 'ibge_atend',
    'paciente':'nome',
    'sexo': 'sexo',
    'idade':'idade',
    'exame': 'laboratorio',
    'data_liberacao':'dt_diag',
    'data_com':'comunicacao',
    'data_1o_sintomas': 'is',
    'evolucao': 'evolucao',
    'data_cura_obito': 'data_evolucao',
    'uf': 'uf_resid'
})

novos_casos = novos_casos[['id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao']]

novos_casos.loc[novos_casos['evolucao']==5,'evolucao'] = 2
novos_casos.loc[~novos_casos['evolucao'].isin([1,2,3]),'evolucao'] = 3

novos_casos['hash'] = (novos_casos['nome'].apply(normalize_hash) +
                  novos_casos['idade'].astype(str) +
                  novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_less'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x - 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_more'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x + 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['identificacao'] = list(range(casosct.iloc[-1,0] + 1,casosct.iloc[-1,0] + 1 + len(novos_casos)))
casosct = casosct.append(novos_casos)
print(len(casosct))
casosct.groupby('evolucao')[['sexo']].count()

In [ ]:
novos_obitos = novos_obitos.rename(columns={
    'id': 'id_notifica',
    'data_cura_obito': 'data_evolucao',
    'paciente':'nome'
})

novos_obitos['hash'] = (novos_obitos['nome'].apply(normalize_hash) +
                          novos_obitos['idade'].astype(str) +
                              novos_obitos['ibge_residencia'].astype(str) )

novos_obitos['data_com_evolucao'] = pd.to_datetime(date.today())
novos_obitos['evolucao'] = 2

novos_obitos = novos_obitos[['hash','id_notifica','evolucao','data_evolucao','data_com_evolucao']]
novos_obitos = novos_obitos.set_index('hash')
casosct = casosct.set_index('hash')
casosct.update(novos_obitos)
casosct.reset_index(inplace=True)
casosct = casosct[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_diag']]

print(len(casosct))
casosct.groupby('evolucao')[['sexo']].count()

In [ ]:
# casosct['is'] = casosct['is'].apply(lambda x: str(x)[:10] if not str(x).lower() in ['none','nat','nan'] else 'none')
# casosct.loc[casosct['is'].str.contains('/'),'is2'] = casosct.loc[casosct['is'].str.contains('/'),'is'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))
# casosct.loc[casosct['is'].str.contains('-'),'is2'] = casosct.loc[casosct['is'].str.contains('-'),'is'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

# casosct['comunicacao'] = casosct['comunicacao'].apply(str)
# casosct.loc[casosct['comunicacao'].str.contains('/'),'com'] = casosct.loc[casosct['comunicacao'].str.contains('/'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))
# casosct.loc[casosct['comunicacao'].str.contains('-'),'com'] = casosct.loc[casosct['comunicacao'].str.contains('-'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

# casosct['data_evolucao'] = casosct['data_evolucao'].apply(str)
# casosct.loc[casosct['data_evolucao'].str.contains('/'),'evo'] = casosct.loc[casosct['data_evolucao'].str.contains('/'),'data_evolucao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='coerce'))
# casosct.loc[casosct['data_evolucao'].str.contains('-'),'evo'] = casosct.loc[casosct['data_evolucao'].str.contains('-'),'data_evolucao'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='coerce'))

# casosct['id_notifica'] = casosct['id_notifica'].apply(int)
# casosct['evolucao'] = casosct['evolucao'].apply(int)
# casosct['is'] = casosct['is2']
# casosct['comunicacao'] = casosct['com']
# casosct['data_evolucao'] = casosct['evo'].apply(lambda x: pd.to_datetime(x))

# casosct.dtypes

In [ ]:
cc.df = casosct
cc.save(f"cc_{today.strftime('%d_%m_%Y')}")
casosct.tail()

In [ ]:
#recuperados

In [ ]:
municipios = Municipios()

macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
municipios['macro'] = municipios['macro'].apply(lambda x: macros_labels_idx[x])

casosct = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios,on=['ibge'],how='left').rename(columns={'ibge':'ibge_resid'})

casosct['municipio_pr'] = casosct['municipio']
casosct.loc[casosct['uf_resid']!='PR','municipio_pr'] = 'Fora'

casosct.loc[casosct['uf_resid']!='PR','ibge7'] = 9999999

casosct['uf_pr'] = 'PR'
casosct.loc[casosct['uf_resid']!='PR','uf_pr'] = 'Fora'

In [ ]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casosct.groupby(['evolucao','uf_pr'])[['sexo']].count().unstack().droplevel(0,1)[['PR','Fora']]
total['GERAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name=0))
total.index = [2,1,3,0]
total = total.sort_index()
total = total.rename(index={0:'CASOS',2:'RECUPERADOS',1:'OBITOS',3:'ATIVOS'})
total.index.name = None
total.columns.name = None

total.astype(float)

In [ ]:
total_macros = casosct.groupby(['macro','evolucao'])[['sexo']].count().unstack()
total_macros.columns = total_macros.columns.droplevel(0)

total_macros['CASOS'] = total_macros.sum(1)
total_macros = total_macros.fillna(0)
total_macros = total_macros.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_macros = total_macros[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_macros.index = [ macros_idx_labels[x] if isinstance(x,int) else x for x in total_macros.index]

total_macros.astype(float)

In [ ]:
total_regionais = casosct.groupby(['rs','regional','evolucao'])[['sexo']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_regionais.astype(float)

In [ ]:
total_municipios = casosct.groupby(['macro','rs','regional','ibge7','municipio_pr','evolucao'])[['sexo']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)

total_municipios

In [ ]:
ministerio_cols = ['rs','municipio_pr','CASOS','OBITOS']
total_municipios.reset_index()[ministerio_cols].sort_values(['rs','municipio_pr']).to_excel(join(default_output,'ministerio.xlsx'),index=False)

In [ ]:
casosct['regional'] = casosct['rs'].apply(lambda x: str(x).zfill(2)) +'ª '+ casosct['regional']
casos_confirmados = casosct[['id_notifica','macro','regional','ibge_resid','municipio','ibge_atend','nome','sexo','idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao','data_evolucao', 'data_com_evolucao']].copy()
casos_confirmados.loc[casos_confirmados['evolucao']==1,'evolucao'] = 'CURA'
casos_confirmados.loc[casos_confirmados['evolucao']==2,'evolucao'] = 'OBITO'
casos_confirmados.loc[casos_confirmados['evolucao']==3,'evolucao'] = 'ATIVO'

In [ ]:
obitos_confirmados = casos_confirmados.loc[casos_confirmados['evolucao']=='OBITO']

In [ ]:
writer = pd.ExcelWriter(join(default_output, f"casos_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx, macro in macros_idx_labels.items():
    casos_macro = casos_confirmados.loc[casos_confirmados['macro']==idx].copy()
    casos_macro['macro'] = macro
    casos_macro.to_excel(writer,macro)
    fit_cols(writer,casos_macro,macro)


writer.save()

In [ ]:
writer = pd.ExcelWriter(join(default_output, f"obitos_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx, macro in macros_idx_labels.items():
    casos_macro = obitos_confirmados.loc[obitos_confirmados['macro']==idx].copy()
    casos_macro['macro'] = macro
    casos_macro.to_excel(writer,macro)
    fit_cols(writer,casos_macro,macro)


writer.save()

In [ ]:
writer = pd.ExcelWriter(join(default_output,f"casos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(casos_confirmados['regional'].unique()):
    casos_regional = casos_confirmados.loc[casos_confirmados['regional']==regional].copy()
    casos_regional['macro'] = casos_regional['macro'].apply(lambda x: macros_idx_labels[x])
    casos_regional.to_excel(writer,regional)
    fit_cols(writer,casos_regional,regional)


writer.save()

In [ ]:
writer = pd.ExcelWriter(join(default_output,f"obitos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(obitos_confirmados['regional'].unique()):
    obitos_regional = obitos_confirmados.loc[obitos_confirmados['regional']==regional].copy()
    obitos_regional['macro'] = obitos_regional['macro'].apply(lambda x: macros_idx_labels[x])
    obitos_regional.to_excel(writer,regional)
    fit_cols(writer,obitos_regional,regional)


writer.save()